In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [7]:
import os
PROJECT_DIRECTORY = os.path.join(os.getcwd(), 'gdrive', 'My Drive','CNN10','checkpoint1.pth')
print(PROJECT_DIRECTORY)
os.makedirs(os.path.dirname(PROJECT_DIRECTORY))

/content/gdrive/My Drive/CNN10/checkpoint1.pth


In [1]:
!pip install torch
!pip install torchvision
!pip install tensorflow

     |████████████████████████████████| 512kB 4.7MB/s 
  Found existing installation: tensorflow-estimator 1.15.0
    Uninstalling tensorflow-estimator-1.15.0:
      Successfully uninstalled tensorflow-estimator-1.15.0


In [8]:
!wget -c https://s3.amazonaws.com/content.udacity-data.com/nd089/Cat_Dog_data.zip
!unzip -qq Cat_Dog_data.zip

--2019-12-30 17:18:47--  https://s3.amazonaws.com/content.udacity-data.com/nd089/Cat_Dog_data.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.96.237
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.96.237|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 580495262 (554M) [application/zip]
Saving to: ‘Cat_Dog_data.zip’

Cat_Dog_data.zip    100%[===================>] 553.60M  26.5MB/s    in 21s     

2019-12-30 17:19:08 (26.1 MB/s) - ‘Cat_Dog_data.zip’ saved [580495262/580495262]



In [9]:
import torchvision
import torch
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms

dataset = torchvision.datasets.ImageFolder(root="./Cat_Dog_data", transform=transforms.Compose([transforms.Resize((64,64)), transforms.ToTensor()]))
def get_subset(indices, start, end):
    return indices[start : start + end]


TRAIN_PCT, VALIDATION_PCT = 0.6, 0.2  # rest will go for test
train_count = int(len(dataset) * TRAIN_PCT)
validation_count = int(len(dataset) * VALIDATION_PCT)
print(validation_count)

indices = torch.randperm(len(dataset))

train_indices = get_subset(indices, 0, train_count)
print(train_indices.shape)
validation_indices = get_subset(indices, train_count, validation_count)
print(validation_indices.shape)
test_indices = get_subset(indices, train_count + validation_count, len(dataset))
print(test_indices.shape)


dataloaders = {
    "train": torch.utils.data.DataLoader(
        dataset, sampler=SubsetRandomSampler(train_indices), batch_size=16 
    ),
    "validation": torch.utils.data.DataLoader(
        dataset, sampler=SubsetRandomSampler(validation_indices),batch_size=16
    ),
    "test": torch.utils.data.DataLoader(
        dataset, sampler=SubsetRandomSampler(test_indices),batch_size=16
    ),
}
torch.utils.data.DataLoader

5000
torch.Size([15000])
torch.Size([5000])
torch.Size([5000])


torch.utils.data.dataloader.DataLoader

In [0]:
import torch.nn as nn
from matplotlib import pyplot as plt
class ConvNet(nn.Module):
    def __init__(self, num_classes=2):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=2, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU())
            
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU())
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=  1),
            nn.BatchNorm2d(64),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU())
        self.layer5 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer6 = nn.Sequential(
            nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU())
        self.layer7 = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU())   
        self.layer8 = nn.Sequential(
            nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU() )
        self.layer9 = nn.Sequential(
            nn.Conv2d(32, 16, kernel_size=3, stride=1, padding=1),
            #torch.nn.Dropout(0.2),
            nn.BatchNorm2d(16),
            nn.ReLU())
        self.fc = nn.Linear(16*8*8, num_classes)
        
    def forward(self, x):
        #print(x.shape)
        out = self.layer1(x)
        #print(out.shape)
        out = self.layer2(out)
       # print(out.shape)
        out=self.layer3(out)
       # print(out.shape)
        out=self.layer4(out)
       # print(out.shape)
        out=self.layer5(out)
        #print(out.shape)
        out=self.layer6(out)
       # print(out.shape)
        out=self.layer7(out)
       # print('out7',out.shape)
        out=self.layer8(out)
        #print('out8',out.shape)
        out=self.layer9(out)
       # print(out.shape)
       
        #print(out.shape)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [13]:
# Device configuration

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
from __future__ import division
# Hyper parameters
num_epochs = 10
num_classes = 2
batch_size = 32
learning_rate = 0.001
model = ConvNet(num_classes).to(device)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
trainaccuracies = []
validationaccuracies = []
# Train the model
total_step = len(dataloaders['train'])
for epoch in range(num_epochs):
    total=0
    correct = 0
    train_acc=0
    train_hist=[]
    for i, (images, labels) in enumerate(dataloaders['train']):
        images = images.to(device)
        labels = labels.to(device)
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        train_hist.append(loss)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
#        if (i+1) % 100 == 0:
#            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
#                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    train_acc=(correct/total)*100
    trainaccuracies.append(train_acc)
    print ('Epoch [{}/{}], Train Loss: {:.4f}, train Acc: {:,.4f}'
                   .format(epoch+1, num_epochs, loss.item(), train_acc))
    model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    with torch.no_grad():
        correct = 0
        total = 0
        val_acc=0
        val_hist=[]
        for images, labels in dataloaders['validation']:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            val_hist.append(loss)
        val_acc=(correct/total)*100
        validationaccuracies.append(val_acc)
        print ('Epoch [{}/{}], Val Loss: {:.4f} , Val Acc: {:,.4f}'
                   .format(epoch+1, num_epochs, loss.item(), val_acc))

torch.save(model.cpu().state_dict(), PROJECT_DIRECTORY) 
        
           

Epoch [1/10], Train Loss: 0.5164, train Acc: 89.9333
Epoch [1/10], Val Loss: 0.1052 , Val Acc: 90.4600
Epoch [2/10], Train Loss: 0.3793, train Acc: 89.9867
Epoch [2/10], Val Loss: 0.3777 , Val Acc: 90.4600
Epoch [3/10], Train Loss: 0.1011, train Acc: 89.9867
Epoch [3/10], Val Loss: 0.1027 , Val Acc: 90.4600
Epoch [4/10], Train Loss: 0.6505, train Acc: 89.9867
Epoch [4/10], Val Loss: 0.3786 , Val Acc: 90.4600
Epoch [5/10], Train Loss: 0.3799, train Acc: 89.9867
Epoch [5/10], Val Loss: 0.3794 , Val Acc: 90.4600
Epoch [6/10], Train Loss: 0.1055, train Acc: 89.9867
Epoch [6/10], Val Loss: 0.1035 , Val Acc: 90.4600
Epoch [7/10], Train Loss: 0.3821, train Acc: 89.9867
Epoch [7/10], Val Loss: 0.3819 , Val Acc: 90.4600
Epoch [8/10], Train Loss: 0.8962, train Acc: 89.9867
Epoch [8/10], Val Loss: 0.3770 , Val Acc: 90.4600
Epoch [9/10], Train Loss: 0.3790, train Acc: 89.9867
Epoch [9/10], Val Loss: 0.1122 , Val Acc: 90.4600
Epoch [10/10], Train Loss: 0.0983, train Acc: 89.9867
Epoch [10/10], Val 